<a href="https://colab.research.google.com/github/rcarasek/Python-for-Investment/blob/main/PyForInvest_Colab_aula08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

In [94]:
import requests

#A url que você quer acesssar
url = 'http://www.fundamentus.com.br/resultado.php'

#Informações para fingir ser um navegador
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
#juntamos tudo com a requests
r = requests.get(url, headers=header)
#E finalmente usamos a função read_html do pandas


In [95]:
df = pd.read_html(r.text,  decimal=',', thousands='.')

In [96]:
type(df)

list

In [97]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [98]:
type(df)

pandas.core.frame.DataFrame

In [99]:
df.head(2)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,MNSA3,0.42,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,0.0,0.0,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9105000.0,-6.52,"-41,11%"
1,POPR4,10.17,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,0.0,0.0,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,545803000.0,0.82,"30,93%"


In [100]:
for coluna in ['Div.Yield'	, 'Mrg Ebit', 'Mrg. Líq.' , 'ROIC' , 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.','')
  df[coluna] = df[coluna].str.replace(',','.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float')/100

In [101]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,MNSA3,0.42,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,-2.0815,-3.6266,3.63,-0.1350,1.4570,0.0,-9.105000e+06,-6.52,-0.4111
1,POPR4,10.17,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0866,0.0565,1.08,0.1525,0.1993,0.0,5.458030e+08,0.82,0.3093
2,PMET3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0410,0.0,-2.908630e+08,0.00,0.3774
3,CFLU4,1000.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0888,0.1072,1.10,0.1768,0.3215,0.0,6.035100e+07,0.06,0.0814
4,CSTB4,147.69,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.0,8.420670e+09,0.14,0.3191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,LWSA3,24.81,735.83,21.55,29.752,0.0011,10.001,40.70,335.64,499.08,329.36,142.72,0.0886,0.0404,1.79,0.0430,0.0293,240037000.0,6.739730e+08,0.23,0.2450
949,PRBC4,14.54,808.37,39.12,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0484,0.0,1.210880e+09,0.00,0.1160
950,UBBR11,14.75,1201.81,3.91,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
951,UBBR3,18.00,1466.61,4.77,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058


### analisar os dados

In [102]:
df= df[df['Liq.2meses'] > 1000000]

In [103]:
df['Liq.2meses']

11     160583000.0
12     296857000.0
13      18716100.0
15      22079900.0
17       1480650.0
          ...     
943      5339110.0
944      1654580.0
947     32977200.0
948    240037000.0
952      2941910.0
Name: Liq.2meses, Length: 270, dtype: float64

In [104]:
df= df[df['Liq.2meses'] > 9000000]

In [105]:
df['Liq.2meses']

11     160583000.0
12     296857000.0
13      18716100.0
15      22079900.0
18      15939000.0
          ...     
938     15557600.0
940    772924000.0
941    115675000.0
947     32977200.0
948    240037000.0
Name: Liq.2meses, Length: 198, dtype: float64

In [106]:
ranking = pd.DataFrame()
ranking['pos'] = range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0 ].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [107]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,CMIN3
1,2,PCAR3,CASH3
2,3,MRFG3,WIZS3
3,4,GOAU4,PSSA3
4,5,WIZS3,ODPV3
...,...,...,...
145,146,SEQL3,SQIA3
146,147,BTOW3,BRML3
147,148,AMAR3,RRRP3
148,149,SQIA3,LOGG3


In [108]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')
b = ranking.pivot_table(columns='ROIC', values='pos')

In [109]:
t = pd.concat([a,b])
t

,ABEV3,AERI3,AGRO3,ALPA4,ALSO3,ALUP11,AMAR3,AMBP3,ANIM3,ASAI3,AURA33,B3SA3,BEEF3,BMOB3,BOAS3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,BTOW3,CAML3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CURY3,CYRE3,DIRR3,DTEX3,ECOR3,...,ROMI3,SAPR11,SAPR4,SBSP3,SEER3,SEQL3,SIMH3,SLCE3,SMLS3,SMTO3,SQIA3,SUZB3,TAEE11,TASA4,TEND3,TGMA3,TIET11,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VAMO3,VIVA3,VIVT3,VVAR3,WEGE3,WIZS3,YDUQ3,ARZZ3,CASH3,LWSA3,MGLU3,RRRP3,SULA11
pos,76,134,43,133,85,7,148.0,99,96,60,35,77,16,114,137,90,53,9,100,113.0,147,37,93,78,84,33,20,17,46,8,12,48,62,26,23,63,92,44,80,40,...,97,10,11,50,38,146,75,69,59,71,149,94,32,15,31,70,22,45,79,58,13,65,95,47,30,28,6,118,119,66,49,130,5,125,NaN,NaN,NaN,NaN,NaN,NaN
pos,38,67,37,28,141,35,NaN,62,111,18,10,23,20,25,72,109,86,21,147,NaN,137,48,104,136,138,87,88,1,53,82,81,46,123,60,12,24,125,122,70,32,...,100,43,42,94,101,144,110,54,112,83,146,99,27,7,61,45,19,95,14,77,36,106,113,16,31,30,8,63,56,119,64,17,3,131,143.0,2.0,135.0,126.0,148.0,96.0


In [110]:
rank = t.dropna(axis=1).sum()
rank

ABEV3    114
AERI3    201
AGRO3     80
ALPA4    161
ALSO3    226
        ... 
VIVT3    185
VVAR3    113
WEGE3    147
WIZS3      8
YDUQ3    256
Length: 144, dtype: int64

In [111]:
rank.sort_values()[:10]

PSSA3      5
WIZS3      8
MRFG3     12
VALE3     14
CMIN3     18
TASA4     22
BRKM5     30
CSNA3     35
BEEF3     36
TIET11    41
dtype: int64

In [112]:
rank.sort_values()[-10:]

NTCO3    263
LINX3    263
POSI3    265
OPCT3    268
ESPA3    270
LOGG3    271
HBSA3    275
BTOW3    284
SEQL3    290
SQIA3    295
dtype: int64